# Model
In this notebook we are defining the model we are going to work with and all the related functions. <br>
Here we can modify the model, add more layers, add more functions if necessary, potentially add loss functions.

In [1]:
import sumolib
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import random

import torch
import torch_geometric
import torch_geometric.data as Data
import torch_geometric.utils as pyg_utils

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import json
import sys
%run ../Codes/Analysis.ipynb

In [3]:
class GNN(nn.Module):
    def __init__(self, edge_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, hidden_dim6, hidden_dim7, hidden_dim8):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(edge_dim, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, hidden_dim3)
        self.conv4 = GCNConv(hidden_dim3, hidden_dim4)
        self.conv5 = GCNConv(hidden_dim4, hidden_dim5)
        self.conv6 = GCNConv(hidden_dim5, hidden_dim6)
        self.conv7 = GCNConv(hidden_dim6, hidden_dim7)
        self.conv8 = GCNConv(hidden_dim7, hidden_dim8)
        self.linear = nn.Linear(hidden_dim8, edge_dim)

    def forward(self, edge_features, edge_index):
        x = self.conv1(edge_features, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        x = F.relu(x)
        x = self.conv6(x, edge_index)
        x = F.relu(x)
        x = self.conv7(x, edge_index)
        x = F.relu(x)
        x = self.conv8(x, edge_index)
        x = F.relu(x)
        x = self.linear(x)
        return x

In [3]:
def training_the_model(model, optimizer, criterion, number_of_epochs, prepared_training_data, prepared_training_data_hiden, prepared_test_data, prepared_test_data_hiden, edges_to_hide, when_to_print_total_loss):
    losses = []
    test_losses = []
    total_train_losses = []
    total_test_losses = []
    total_geh_losses = []

    for epoch in range(number_of_epochs):
        model.train()
        optimizer.zero_grad()
        i = random.randint(0, len(prepared_training_data)-1)
        """ if epoch % 100 == 0:
            i = random.randint(0, len(prepared_test_data)-1) """
        # Forward pass
        output = model(prepared_training_data_hiden[i].edge_attr, prepared_training_data_hiden[i].edge_index)
        
        # Compute loss using the predicted features and the input features of the hidden edges
        loss = criterion(output[edges_to_hide], prepared_training_data[i].edge_attr[edges_to_hide])
        losses.append(float(loss))
        loss.backward()
        optimizer.step()

        i = random.randint(0, len(prepared_test_data)-1)
        """ if epoch % 100 == 0:
            i = random.randint(0, len(prepared_test_data)-1) """
        # Forward pass
        output_test = model(prepared_test_data_hiden[i].edge_attr, prepared_test_data_hiden[i].edge_index)
        
        # Compute loss using the predicted features and the input features of the hidden edges
        loss_test = criterion(output_test[edges_to_hide], prepared_test_data[i].edge_attr[edges_to_hide])
        test_losses.append(float(loss_test))
        if epoch % when_to_print_total_loss == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()} , Test loss: {loss_test.item()} ')
            total_train_loss = evaluate_all_graphs(model, prepared_training_data_hiden, prepared_training_data, edges_to_hide, criterion)
            total_train_losses.append(float(total_train_loss))

            total_test_loss = evaluate_all_graphs(model, prepared_test_data_hiden, prepared_test_data, edges_to_hide, criterion)
            total_test_losses.append(float(total_test_loss))

            #total_geh_loss = evaluate_all_graphs_GEH(model, prepared_test_data_hiden, prepared_test_data, edges_to_hide)
            #total_geh_losses.append(float(total_geh_loss))



    return losses, total_train_losses, total_test_losses , test_losses, total_geh_losses

In [ ]:
def training_the_model_forecasting(model, optimizer, criterion, number_of_epochs, prepared_training_data, prepared_training_data_hidden, prepared_test_data, prepared_test_data_hidden, edges_to_hide, when_to_print_total_loss, time_step):
    losses = []
    test_losses = []
    total_train_losses = []
    total_test_losses = []
    total_geh_losses = []

    for epoch in range(number_of_epochs):
        model.train()
        optimizer.zero_grad()
        #Choosing a graph randomly
        which_list = random.randint(0,len(prepared_training_data_hidden)-1)
        which_position = random.randint(0,len(prepared_training_data_hidden[which_list])-1-time_step)
        #Forward pass
        output = model(prepared_training_data_hidden[which_list][which_position].edge_attr, prepared_training_data_hidden[which_list][which_position].edge_index)

        #Calculating loss
        loss = criterion(output, prepared_training_data[which_list][which_position+time_step].edge_attr)
        losses.append(float(loss))
        loss.backward()
        optimizer.step()

        which_list = random.randint(0,len(prepared_test_data_hidden)-1)
        which_position = random.randint(0,len(prepared_test_data_hidden[which_list])-1-time_step)
        
        output_test = model(prepared_test_data_hidden[which_list][which_position].edge_attr, prepared_test_data_hidden[which_list][which_position].edge_index)
        loss_test = criterion(output_test, prepared_test_data[which_list][which_position+time_step].edge_attr)
        test_losses.append(float(loss_test))

        if epoch % when_to_print_total_loss == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()} , Test loss: {loss_test.item()} ')
            total_train_loss, total_test_loss = evaluate_all_graphs_forecasting(model, prepared_training_data, prepared_training_data_hidden ,prepared_test_data, prepared_test_data_hidden, edges_to_hide, criterion, time_step)
            total_train_losses.append(float(total_train_loss))
            total_test_losses.append(float(total_test_loss))

            #total_geh_loss = evaluate_all_graphs_GEH(model, prepared_test_data_hiden, prepared_test_data, edges_to_hide)
            #total_geh_losses.append(float(total_geh_loss))



    return losses, total_train_losses, total_test_losses , test_losses, total_geh_losses